This code prepares the data for EDA. It takes around 20-30mins to run as it reads multiple .CSV files.

In [ ]:
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Function to create CCI, RSI and MACD

def indicators(data):
    ############################################# CCI #############################################################
    # Calculate the Typical Price
    data['Typical Price'] = (data['HIGH'] + data['LOW'] + data['CLOSE']) / 3
    
    # Define the number of days to use in the SMA and Mean Deviation calculations
    n = 34
    
    # Calculate the Simple Moving Average (SMA)
    data['SMA'] = data['Typical Price'].rolling(n).mean()
    
    # Calculate the Mean Deviation
    data['Mean Deviation'] = np.abs(data['Typical Price'] - data['SMA']).rolling(n).mean()
    
    # Calculate the Commodity Channel Index (CCI)
    data['CCI'] = (data['Typical Price'] - data['SMA']) / (0.015 * data['Mean Deviation'])
    
    # Print the last 10 rows of the data with the CCI column
    # print(data.tail(10))
    data = data.drop_duplicates()
    
    print('CCI:',len(data))
    
    ############################################# RSI #############################################################
    # Set time period for RSI calculation
    period = 14
    
    # Calculate price difference
    delta = data['CLOSE'].diff()
    
    # Create dataframe to store RSI values
    rsi_df = pd.DataFrame()
    
    # Calculate gain and loss
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gain and loss for time period
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Add RSI values to dataframe
    rsi_df['TIMESTAMP'] = data['TIMESTAMP']
    rsi_df['RSI'] = rsi
    
    rsi_df = rsi_df.drop_duplicates()
    print('RSI:',len(rsi_df))
    
    ############################################# MACD #############################################################
    # FOR NIFTY BEES s 10 long 21
    # Set time periods for EMA calculation
    short_period = 12
    long_period = 26
    signal_period = 9
    
    # Calculate short and long period EMAs
    short_ema = data['CLOSE'].ewm(span=short_period, adjust=False).mean()
    long_ema = data['CLOSE'].ewm(span=long_period, adjust=False).mean()
    
    # Calculate MACD line
    macd_line = short_ema - long_ema
    
    # Calculate signal line
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    
    # Calculate MACD histogram
    macd_hist = macd_line - signal_line
    
    # Create dataframe to store MACD values
    macd_df = pd.DataFrame()
    
    # Add MACD values to dataframe
    macd_df['TIMESTAMP'] = data['TIMESTAMP']
    macd_df['MACD'] = macd_line
    macd_df['Signal'] = signal_line
    macd_df['Histogram'] = macd_hist
    
    # Display dataframe
    print('MACD:',len(macd_df))
    
    # Merge CCI and RSI
    rsi_cci = data.merge(rsi_df,on ='TIMESTAMP',how= 'inner')
    rsi_cci_macd = rsi_cci.merge(macd_df,on ='TIMESTAMP',how= 'inner')
    
    print('Combined:',len(rsi_cci_macd))
    
    return rsi_cci_macd

In [3]:
'''
This function extracts the Stocks from all .csv files
'''
def extarct_Stocks(stocks_to_extract_list, columns_to_extract, csv_directory):
    '''
    Input:
        stocks_to_extract_list -> List of all the stocks you want to extract
        columns_to_extract -> The four columns we are interested in
        csv_directory -> Where the Bhavcopy csv files are stored
    Output:
        It stores the csv of the stocks in the same directory as the program
    '''
    combined_df = pd.DataFrame()
    for stocks_to_extract in stocks_to_extract_list:

        result_df = pd.DataFrame()

        for filename in os.listdir(csv_directory):
            if filename.endswith('.csv'):
                temp = os.path.join(csv_directory, filename)
                df = pd.read_csv(temp)

                df_stock = df.loc[df['SYMBOL'].isin([stocks_to_extract]), columns_to_extract]

                result_df = pd.concat([result_df, df_stock])
                
        
        # Load the data from the bhavcopy file
        data = result_df.copy()
        
        
        print('Read:',len(data))
        #Remove reduce
        data= data[data['TOTALTRADES']>=50]

        # Convert the 'TIMESTAMP' column to datetime objects
        data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%d-%b-%Y')
            
        
        # Sort the data by the 'TIMESTAMP' column in ascending order
        data = data.sort_values('TIMESTAMP')
        
        data = data.drop_duplicates()
 
        result_df_final = indicators(data)
        combined_df = pd.concat([result_df_final, combined_df])
    return combined_df

In [4]:
#cd C:/UW/WI23/BIOSTAT557/Project/

C:\UW\WI23\BIOSTAT557\Project


In [5]:
if __name__ == "__main__":
    
    # csv with top 100 stocks
    top100 = pd.read_csv('ind_nifty100list.csv')
    stocks_to_extract_list = top100['Symbol'].unique()
    
    # Enter column to get
    columns_to_extract = ['TIMESTAMP','SYMBOL','OPEN', 'HIGH', 'LOW', 'CLOSE','LAST','PREVCLOSE','TOTTRDQTY','TOTTRDVAL','TOTALTRADES']
    # Enter directory to the Bhavcopy folder
    csv_directory = '/Bhavcopy'
    df = extarct_Stocks(stocks_to_extract_list, columns_to_extract, csv_directory)
    
    

Read: 1735
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1127
CCI: 1121
RSI: 1121
MACD: 1121
Combined: 1121
Read: 1736
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 494
CCI: 488
RSI: 488
MACD: 488
Combined: 488
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1737
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1738
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1740
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1440
CCI: 1430
RSI: 1430
MACD: 1430
Combined: 1430
Read: 1742
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1735
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1738
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1740
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1729
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1182
CCI: 1177
RSI: 1177
MACD: 1177
Combined: 1177
Read: 1729
CCI: 1729
RSI: 1729
MACD: 1729
Combined: 1729
Read: 1738
CCI: 1729
RSI: 1729
MACD:

In [9]:
df

,TIMESTAMP,SYMBOL,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,Typical Price,SMA,Mean Deviation,CCI,RSI,MACD,Signal,Histogram
0,2016-01-01,NIFTYBEES,802.00,807.50,800.00,805.06,804.00,803.01,27347,2.199696e+07,1020,804.186667,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1,2016-01-04,NIFTYBEES,802.95,803.50,789.00,790.31,790.40,805.06,40397,3.210131e+07,1818,794.270000,NaN,NaN,NaN,NaN,-1.176638,-0.235328,-0.941311
2,2016-01-05,NIFTYBEES,793.87,793.95,787.95,789.81,790.00,790.31,34797,2.750461e+07,1000,790.570000,NaN,NaN,NaN,NaN,-2.124983,-0.613259,-1.511724
3,2016-01-06,NIFTYBEES,789.80,790.88,783.70,785.16,784.40,789.81,48620,3.829565e+07,1131,786.580000,NaN,NaN,NaN,NaN,-3.214712,-1.133549,-2.081162
4,2016-01-07,NIFTYBEES,782.01,783.00,767.25,768.92,768.00,785.16,156360,1.205742e+08,3450,773.056667,NaN,NaN,NaN,NaN,-5.327351,-1.972310,-3.355042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,2022-12-26,ACC,2372.00,2448.50,2360.00,2423.65,2429.15,2374.10,480611,1.165814e+09,33035,2410.716667,2532.664216,117.982771,-68.906981,28.940073,-1.850621,30.858689,-32.709310
1725,2022-12-27,ACC,2430.95,2487.95,2428.10,2478.00,2486.00,2423.65,260633,6.408237e+08,17590,2464.683333,2532.916667,117.570026,-38.690890,37.407735,-6.263760,23.434199,-29.697960
1726,2022-12-28,ACC,2475.00,2477.00,2447.55,2455.35,2456.00,2478.00,201331,4.953475e+08,10875,2459.966667,2534.337255,118.382656,-41.881466,33.905704,-11.456804,16.455999,-27.912803
1727,2022-12-29,ACC,2455.00,2467.70,2411.15,2447.85,2446.70,2455.35,352346,8.594227e+08,25719,2442.233333,2535.506863,119.860107,-51.879107,33.531613,-15.993157,9.966168,-25.959325


In [13]:
# Remove stocks with insufficient data along with NIFTY BEES
# Filter out records with the specified values in the 'SYMBOL' column
symbols_to_remove = ['BRITANNIA', 'HDFC', 'JSWSTEEL', 'MUTHOOTFIN', 'NTPC', 'SBIN', 'UPL', 'VEDL', 'KOTAKBANK','NIFTYBEES']
df = df[~df['SYMBOL'].isin(symbols_to_remove)]
df.to_csv('Top100stocks_indicator.csv',index= False)